<a href="https://colab.research.google.com/github/LukasFeldler/SemAI23/blob/main/UE05_LuFe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Course Instructor: Bernd Neumayr, JKU

# UE05: SHACL

Complete the **10 tasks (1 point per task)** in the `4. SHACL` sheet of `SemAI.jar` first and then transfer them to this notebook.

For each task include:
- A headline including the task number
- The task description 
- The data graph and your solution (the shapes graph) in executable form
- After executing the validation, print out the validation results in tabular form.  


In [ ]:
# Install required packages in the current Jupyter kernel
!pip install -q rdflib 
!pip3 install -q pyshacl

In [ ]:
# Imports
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace, Dataset
from rdflib.namespace import FOAF , XSD , RDFS 
from rdflib.plugins.sparql.processor import SPARQLResult
from rdflib.namespace import NamespaceManager

from pyshacl import validate

import pandas as pd

def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = [ { var : res[var].n3(graph.namespace_manager) if (isinstance(res[var],URIRef) and use_prefixes) else res[var] for var in results.vars } for res in results ]     
                                        # construct a list of dictionaries, as intermediate format to construct the pandas DataFrame, use prefixes to abbreviate URIs                
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def validation_report_as_dataframe(validation_report):
  df = sparql_select(results_graph,"""
		SELECT  ?focusNode ?resultPath ?value ?sourceConstraintComponent ?sourceShape ?resultMessage
		WHERE
  		{ ?vr	a sh:ValidationResult ;
						sh:focusNode ?focusNode ;
						sh:sourceConstraintComponent ?sourceConstraintComponent ;
						sh:sourceShape ?sourceShape ;
						sh:resultMessage ?resultMessage .					 
				OPTIONAL { ?vr sh:value ?value . }
				OPTIONAL { ?vr sh:resultPath ?resultPath . }
  		}
  """,use_prefixes=True)
  return df

def shacl_validate(dg,sg):
  return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)  
  

def shacl_validate_with_rules(dg,sg):
	return validate(dg,shacl_graph=sg,
      inference='rdfs',
      abort_on_first=False,
      allow_infos=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=True,
      iterate_rules=True, inplace=True,
      js=False,
      debug=False)

Task 1
Wenn eine Person eine x:knows Beziehung hat, dann sollte das Ziel dieser Beziehung als IRI angegeben sein.

In [ ]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Peter>  rdf:type  <Person> ;
        :knows    <John> ;
        :knows    [ :name  "Peter" ] .

<John>  :knows  [ :name   "Mary" ] .     
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

:PersonShape a sh:NodeShape ;
    sh:targetClass <Person> ;
    sh:property <PersonKnowsShape> .

:PersonKnowsShape
    a sh:PropertyShape;
    sh:path :knows ;
    sh:nodeKind sh:IRI.
    
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Peter ;
            sh:resultMessage "Value is not of Node Kind sh:IRI" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeKindConstraintComponent ;
            sh:sourceShape :PersonKnowsShape ;
            sh:value [ a rdfs:Resource ;
                    :name "Peter" ] ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Peter,:knows,naf85dec874044a3e84d798e057b9025cb1,sh:NodeKindConstraintComponent,:PersonKnowsShape,Value is not of Node Kind sh:IRI


Task 2
Die Klassen Man und Woman sind disjunkt. Hinweis: verwenden Sie sh:not.

In [ ]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<YoungMan>  rdfs:subClassOf  <Man> .

<John>  rdf:type  <YoungWoman> , <YoungMan> .

<Mary>  rdf:type  <Woman> .

<Peter>  rdf:type  <Man> .

<YoungWoman>  rdfs:subClassOf  <Woman> .

<Jane>  rdf:type  <YoungWoman> .

<Susi>  rdf:type  <YoungWoman> , <Man> .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

	:ManShape a sh:NodeShape;
sh:targetClass <Man>;
sh:not [      
   sh:class <Woman>
].

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :John ;
            sh:resultMessage "Node :John conforms to shape [ sh:class :Woman ]" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NotConstraintComponent ;
            sh:sourceShape :ManShape ;
            sh:value :John ],
        [ a sh:ValidationResult ;
            sh:focusNode :Susi ;
            sh:resultMessage "Node :Susi conforms to shape [ sh:class :Woman ]" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NotConstraintComponent ;
            sh:sourceShape :ManShape ;
            sh:value :Susi ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Susi,None,:Susi,sh:NotConstraintComponent,:ManShape,Node :Susi conforms to shape [ sh:class :Woman ]
1,:John,None,:John,sh:NotConstraintComponent,:ManShape,Node :John conforms to shape [ sh:class :Woman ]


Task 3
Wenn eine Person eine :knows Beziehung hat, dann muss das Ziel dieser Beziehung eine Person sein.

In [ ]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Peter>  rdf:type  <Person> ;
        :knows    <Mary> ;
        :knows    [ :name  "Mary" ] ;
        :knows    [ rdf:type  <Person> ] .

<Jane>  :knows  <Jim> .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

:PersonShape a sh:NodeShape ;
    sh:targetClass <Person> ;
    sh:property <PersonKnowsShape> .

:PersonKnowsShape
    sh:path :knows ;
    sh:class <Person> .

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Peter ;
            sh:resultMessage "Value does not have class :Person" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :PersonKnowsShape ;
            sh:value [ a rdfs:Resource ;
                    :name "Mary" ] ],
        [ a sh:ValidationResult ;
            sh:focusNode :Peter ;
            sh:resultMessage "Value does not have class :Person" ;
            sh:resultPath :knows ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :PersonKnowsShape ;
            

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Peter,:knows,nda8db2838a17477a8922e2937600a62ab1,sh:ClassConstraintComponent,:PersonKnowsShape,Value does not have class :Person
1,:Peter,:knows,:Mary,sh:ClassConstraintComponent,:PersonKnowsShape,Value does not have class :Person


Task 4
Jede Person hat genau ein Alter (:age). Eine Person ist mindestens 0 und maximal 150 Jahre alt. Erwachsene (Klasse Adult) sind mindestens 19 Jahre alt. Senioren sind mindestens 65 Jahre alt. Kinder sind maximal 12 Jahre alt.
Verwenden Sie sh:minInclusive und sh:maxInclusive (siehe SHACL Recommendation).

In [ ]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<ABC>   :age    -1 , 27 , 197 .

<John>  rdf:type  <Person> ;
        :age      151 .

<Adult>  rdfs:subClassOf  <Person> .

<Beny>  rdf:type  <Child> ;
        :age      13 .

<Zoe>   rdf:type  <Child> ;
        :age      12 .

<Mary>  rdf:type  <Woman> ;
        :age      17 .

<Bibi>  rdf:type  <Person> ;
        :age      0 .

<Peter>  rdf:type  <Person> .

<Senior>  rdfs:subClassOf  <Adult> .

<Jane>  rdf:type  <Woman> ;
        :age      67 .

<Lili>  rdf:type  <Person> ;
        :age      -1 .

<Senior1>  rdf:type  <Senior> ;
        :age      65 .

<Child>  rdfs:subClassOf  <Person> .

<Bibo>  rdf:type  <Person> ;
        :age      34 , 12 .

<Woman>  rdfs:subClassOf  <Adult> .

<Senior2>  rdf:type  <Senior> ;
        :age      64 .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

:PersonShape a sh:NodeShape ;
    sh:targetClass <Person> ;
    sh:property <PersonAgeShape> .

:PersonAgeShape
    sh:path :age;
    sh:minInclusive 0 ;
    sh:maxInclusive 150 ;
    sh:maxCount 1 ;
    sh:minCount 1 .


:AdultShape a sh:NodeShape ;
    sh:targetClass <Adult> ;
    sh:property <AdultAgeShape> .

:AdultAgeShape
    sh:path :age ;
    sh:minInclusive 19 .

:SeniorShape a sh:NodeShape ;
    sh:targetClass <Senior> ;
    sh:property <SeniorAgeShape> .

:SeniorAgeShape
    sh:path :age ;
    sh:minInclusive 65 .

:ChildShape a sh:NodeShape ;
    sh:targetClass <Child> ;
    sh:property <ChildAgeShape> .

:ChildAgeShape
    sh:path :age ;
    sh:maxInclusive 12 .
    
""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Mary ;
            sh:resultMessage "Value is not >= Literal(\"19\", datatype=xsd:integer)" ;
            sh:resultPath :age ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MinInclusiveConstraintComponent ;
            sh:sourceShape :AdultAgeShape ;
            sh:value 17 ],
        [ a sh:ValidationResult ;
            sh:focusNode :Senior2 ;
            sh:resultMessage "Value is not >= Literal(\"65\", datatype=xsd:integer)" ;
            sh:resultPath :age ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MinInclusiveConstraintComponent ;
            sh:sourceShape :SeniorAgeShape ;
            sh:value 64 ],
        [ a sh:ValidationResult ;
            s

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Bibo,:age,None,sh:MaxCountConstraintComponent,:PersonAgeShape,More than 1 values on :Bibo->:age
1,:Peter,:age,None,sh:MinCountConstraintComponent,:PersonAgeShape,Less than 1 values on :Peter->:age
2,:John,:age,151,sh:MaxInclusiveConstraintComponent,:PersonAgeShape,"Value is not <= Literal(""150"", datatype=xsd:in..."
3,:Lili,:age,-1,sh:MinInclusiveConstraintComponent,:PersonAgeShape,"Value is not >= Literal(""0"", datatype=xsd:inte..."
4,:Mary,:age,17,sh:MinInclusiveConstraintComponent,:AdultAgeShape,"Value is not >= Literal(""19"", datatype=xsd:int..."
5,:Beny,:age,13,sh:MaxInclusiveConstraintComponent,:ChildAgeShape,"Value is not <= Literal(""12"", datatype=xsd:int..."
6,:Senior2,:age,64,sh:MinInclusiveConstraintComponent,:SeniorAgeShape,"Value is not >= Literal(""65"", datatype=xsd:int..."


Task 5
Jede Person hat maximal einen Namen. Der Name muss ein Literal sein und vom Datentyp String. Personen dürfen abgesehen von :name und rdf:type keine weiteren Eigenschaften haben.

In [ ]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<John>  rdf:type  <MalePerson> ;
        :knows    <Mary> .

<FemalePerson>  rdfs:subClassOf  <Person> .

<Mary>  rdf:type  <FemalePerson> ;
        :name     "Mary" .

<Bibi>  rdf:type  <Person> ;
        :name     "Jim" , "Bibi" .

<Jane>  rdf:type  <Person> ;
        :name     2343 .

<Jim>   rdf:type  <Person> ;
        :name     <Jim> .

<MalePerson>  rdfs:subClassOf  <Person> .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

:PersonShape a sh:NodeShape ;
    sh:targetClass <Person> ;
    sh:property <NameShape> ;
    sh:closed true;
    sh:ignoredProperties ( rdf:type ) .

:NameShape
    sh:path :name;
    sh:datatype xsd:string;
    sh:maxCount 1 ;
    sh:nodeKind sh:Literal .

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Bibi ;
            sh:resultMessage "More than 1 values on :Bibi->:name" ;
            sh:resultPath :name ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:MaxCountConstraintComponent ;
            sh:sourceShape :NameShape ],
        [ a sh:ValidationResult ;
            sh:focusNode :Jim ;
            sh:resultMessage "Value is not of Node Kind sh:Literal" ;
            sh:resultPath :name ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:NodeKindConstraintComponent ;
            sh:sourceShape :NameShape ;
            sh:value :Jim ],
        [ a sh:ValidationResult ;
            sh:focusNode :Jane ;
            sh:resultMessage "Value is not Literal with dat

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Jim,:name,:Jim,sh:DatatypeConstraintComponent,:NameShape,Value is not Literal with datatype xsd:string
1,:Jim,:name,:Jim,sh:NodeKindConstraintComponent,:NameShape,Value is not of Node Kind sh:Literal
2,:Jane,:name,2343,sh:DatatypeConstraintComponent,:NameShape,Value is not Literal with datatype xsd:string
3,:Bibi,:name,None,sh:MaxCountConstraintComponent,:NameShape,More than 1 values on :Bibi->:name
4,:John,:knows,:Mary,sh:ClosedConstraintComponent,:PersonShape,Node :John is closed. It cannot have value: :Mary


Task 6
Städte sind via Property :inCountry Ländern zugeordnet. Europäische Städte sind europäischen Ländern zugeordnet. Österreichische Städte sind Austria zugeordnet.

In [ ]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Paris>  rdf:type   <EuropeanCity> ;
        :inCountry  <France> .

<AustrianCity>  rdfs:subClassOf  <EuropeanCity> .

<Salzburg>  rdf:type  <AustrianCity> ;
        :inCountry  <Germany> .

<EuropeanCountry>  rdfs:subClassOf  <Country> .

<NewYork>  rdf:type  <City> ;
        :inCountry  <USA> .

<Germany>  rdf:type  <EuropeanCountry> .

<EuropeanCity>  rdfs:subClassOf  <City> .

<Vienna>  rdf:type  <AustrianCity> ;
        :inCountry  <Austria> .

<Austria>  rdf:type  <EuropeanCountry> .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

:CityShape a sh:NodeShape ;
    sh:targetClass <City> ;
    sh:property <InCountryShape> .

:InCountryShape
    sh:path :inCountry;
    sh:class <Country> .

:EuropeanCityShape a sh:NodeShape ;
    sh:targetClass <EuropeanCity>;
    sh:property <InEuropeanCountryShape> .

:InEuropeanCountryShape
    sh:path :inCountry ;
    sh:class <EuropeanCountry> .

:AustrianCityShape a sh:NodeShape ;
    sh:targetClass <AustrianCity> ;
    sh:property <InAustriaShape> .

:InAustriaShape
    sh:path :inCountry ;
    sh:in (<Austria>) ;
    sh:closed true ;
    sh:ignoredProperties (rdf:type :inCountry) .

""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Paris ;
            sh:resultMessage "Value does not have class :Country" ;
            sh:resultPath :inCountry ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :InCountryShape ;
            sh:value :France ],
        [ a sh:ValidationResult ;
            sh:focusNode :Paris ;
            sh:resultMessage "Value does not have class :EuropeanCountry" ;
            sh:resultPath :inCountry ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :InEuropeanCountryShape ;
            sh:value :France ],
        [ a sh:ValidationResult ;
            sh:focusNode :S

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:NewYork,:inCountry,:USA,sh:ClassConstraintComponent,:InCountryShape,Value does not have class :Country
1,:Paris,:inCountry,:France,sh:ClassConstraintComponent,:InCountryShape,Value does not have class :Country
2,:Salzburg,:inCountry,:Germany,sh:InConstraintComponent,:InAustriaShape,Value :Germany not in list [':Austria']
3,:Paris,:inCountry,:France,sh:ClassConstraintComponent,:InEuropeanCountryShape,Value does not have class :EuropeanCountry


Task 7
Die Property :worksFor darf nur Personen (als Subjekt) und Organisation (als Objekt) miteinander verbinden.
Hinweis: Lösen Sie die Aufgabe ohne Property Shapes.

In [ ]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Mary>  :worksFor  <JKU> .

<Jane>  rdf:type   <Person> ;
        :worksFor  <LMU> .

<Jim>   rdf:type   <Man> ;
        :worksFor  <LMU> .

<University>  rdfs:subClassOf  <Organization> .

<Bob>   rdf:type   <Person> ;
        :worksFor  [ rdf:type  <Organization> ] .

<JKU>   rdf:type  <University> .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

:WorksForSubjectShape
    sh:targetSubjectsOf :worksFor ;
    sh:class <Person> .

:WorksForObjectShape
    sh:targetObjectsOf :worksFor ;
    sh:class <Organization> .





""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Mary ;
            sh:resultMessage "Value does not have class :Person" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :WorksForSubjectShape ;
            sh:value :Mary ],
        [ a sh:ValidationResult ;
            sh:focusNode :LMU ;
            sh:resultMessage "Value does not have class :Organization" ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:ClassConstraintComponent ;
            sh:sourceShape :WorksForObjectShape ;
            sh:value :LMU ],
        [ a sh:ValidationResult ;
            sh:focusNode :Jim ;
            sh:resultMessage "Value does not have class :Person" ;
            sh:

,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Jim,None,:Jim,sh:ClassConstraintComponent,:WorksForSubjectShape,Value does not have class :Person
1,:Mary,None,:Mary,sh:ClassConstraintComponent,:WorksForSubjectShape,Value does not have class :Person
2,:LMU,None,:LMU,sh:ClassConstraintComponent,:WorksForObjectShape,Value does not have class :Organization


Task 8
Peter kennt ausschließlich Männer, die in Österreich leben.

Task 9
Das Nettogewicht eines Produkts darf nicht größer sein als das Bruttogewicht.
Verwenden Sie sh:lessThanOrEquals (siehe SHACL Recommendation).

In [ ]:
dg = Graph() # the Data Graph
dg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

<Cookies>  rdf:type   <Product> ;
        :grossWeight  0.2 ;
        :netWeight    0.12 .

<Milk>  rdf:type      <Product> ;
        :grossWeight  1.1 ;
        :netWeight    1 .

<Peter>  rdf:type     <Person> ;
        :grossWeight  74 ;
        :netWeight    72 .

<Bread>  rdf:type     <Product> ;
        :grossWeight  1.1 ;
        :netWeight    1.2 .
""")

sg = Graph() # the Shapes Graph
sg.parse(format="turtle", data="""
PREFIX  rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  sh:    <http://www.w3.org/ns/shacl#>
PREFIX  xsd:   <http://www.w3.org/2001/XMLSchema#>
PREFIX  :      <http://example.org/>
BASE   	       <http://example.org/>

:ProductShape a sh:NodeShape ;
    sh:targetClass <Product> ;
    sh:property [
       sh:path :netWeight ;
       sh:lessThanOrEquals :grossWeight
].




""")

conforms, results_graph, results_text = shacl_validate(dg,sg)  

if conforms:
	print("everything good")
else:
	print(results_graph.serialize(format='turtle'))

validation_report_as_dataframe(results_graph)

@prefix : <http://example.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

[] a sh:ValidationReport ;
    sh:conforms false ;
    sh:result [ a sh:ValidationResult ;
            sh:focusNode :Bread ;
            sh:resultMessage "Value of :Bread->:grossWeight < Literal(\"1.2\", datatype=xsd:decimal)" ;
            sh:resultPath :netWeight ;
            sh:resultSeverity sh:Violation ;
            sh:sourceConstraintComponent sh:LessThanOrEqualsConstraintComponent ;
            sh:sourceShape [ sh:lessThanOrEquals :grossWeight ;
                    sh:path :netWeight ] ;
            sh:value 1.2 ] .




,focusNode,resultPath,value,sourceConstraintComponent,sourceShape,resultMessage
0,:Bread,:netWeight,1.2,sh:LessThanOrEqualsConstraintComponent,n8d7843c99ab34505b85b3a4e63f5dacfb1,"Value of :Bread->:grossWeight < Literal(""1.2"",..."


Task 10
Die Objekte von knows-Statements haben einen Namen (Property :name) oder haben einen Vornamen (:givenName) und einen Nachnamen (:familyName). eine